In [4]:
# coding:utf-8
from igraph import *
import csv

In [6]:
g = Graph([(0, 1), (0, 2), (1, 2)])
g

In [7]:
g.degree()

[2, 2, 2]

In [8]:
summary(g)

IGRAPH U--- 3 3 -- 


In [5]:
# 度
netPath = r'E:\python\PythonSpace\Git\Python\Igraph\Data\net.data'
edges = []
with open(netPath, 'r') as fr:
	for row in csv.reader(fr.read().splitlines()):
		u, v = [i for i in row]
		edges.append((u, v))
# 默认生成无向图
# vertex_name_attr设置点的名字
g = Graph.TupleList(edges, vertex_name_attr="point")
print(g)
# g.vs表示点的顺序
for p in g.vs:
	print(type(p["point"]), p["point"], p.degree())

IGRAPH U--- 14 17 --
+ attr: point (v)
+ edges:
0--1 0--2 1--2 2--3 4--5 4--6 5--6 3--6 3--7 7--8 8--9 8--10 9--10 7--11
11--12 11--13 12--13
<class 'str'> 1 2
<class 'str'> 2 2
<class 'str'> 3 3
<class 'str'> 7 3
<class 'str'> 4 2
<class 'str'> 5 2
<class 'str'> 6 3
<class 'str'> 8 3
<class 'str'> 9 3
<class 'str'> 10 2
<class 'str'> 11 2
<class 'str'> 12 3
<class 'str'> 13 2
<class 'str'> 14 2


In [22]:
for p in g.vs:
	print(p)

igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 0, {'point': '1'})
igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 1, {'point': '2'})
igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 2, {'point': '3'})
igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 3, {'point': '7'})
igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 4, {'point': '4'})
igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 5, {'point': '5'})
igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 6, {'point': '6'})
igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 7, {'point': '8'})
igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 8, {'point': '9'})
igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 9, {'point': '10'})
igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 10, {'point': '11'})
igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 11, {'point': '12'})
igraph.Vertex(<igraph.Graph object at 0x0000023400E2D8B8>, 

In [28]:
# 紧密中心性
# 表示某个节点到达其它节点的难易程度，紧密中心性越大，则说明该节点
# 周围的其它点越密集
# 计算方式：该节点到达其它所有节点（不包括自身）的最短路径和的倒数
# 计算“7”号节点的紧密中心度
# get_all_shortest_paths(3)取第四个元素
paths = g.get_all_shortest_paths(7)
points = g.vs["point"]
pathLength = 0
for p in paths:
	print([points[x] for x in p])
	# “7”号节点到其它节点的路径长度
	pathLength += len(p) - 1
print("closeness centrality =", (len(paths)-1)/pathLength)

['8', '7', '3', '1']
['8', '7', '3', '2']
['8', '7', '3']
['8', '7']
['8', '7', '6', '4']
['8', '7', '6', '5']
['8', '7', '6']
['8']
['8', '9']
['8', '9', '10']
['8', '9', '11']
['8', '12']
['8', '12', '13']
['8', '12', '14']
closeness centrality = 0.48148148148148145


In [26]:
# 使用紧密中心性函数
ccvs = []
for p in zip(g.vs, g.closeness()):
	ccvs.append({"point" : p[0]["point"], "cc" : p[1]})
# 逆序输出前10个
sorted(ccvs, key=lambda k : k["cc"], reverse=True)[:10]

[{'cc': 0.48148148148148145, 'point': '7'},
 {'cc': 0.48148148148148145, 'point': '8'},
 {'cc': 0.37142857142857144, 'point': '3'},
 {'cc': 0.37142857142857144, 'point': '6'},
 {'cc': 0.37142857142857144, 'point': '9'},
 {'cc': 0.37142857142857144, 'point': '12'},
 {'cc': 0.2826086956521739, 'point': '1'},
 {'cc': 0.2826086956521739, 'point': '2'},
 {'cc': 0.2826086956521739, 'point': '4'},
 {'cc': 0.2826086956521739, 'point': '5'}]

In [29]:
for p in paths:
	print(p)

[7, 3, 2, 0]
[7, 3, 2, 1]
[7, 3, 2]
[7, 3]
[7, 3, 6, 4]
[7, 3, 6, 5]
[7, 3, 6]
[7]
[7, 8]
[7, 8, 9]
[7, 8, 10]
[7, 11]
[7, 11, 12]
[7, 11, 13]


In [32]:
# 介数中心性
# 用来衡量一个节点作为“桥梁”的重要程度
# 介数的计算方式：
#   对于任意一个点p，首先计算整个图中任意两个顶点之间最短路径数量之和，
#   然后判断哪些路径中（不能是头尾节点）包含节点p，包含则p点的介数加1
sp = []
point = 7
for i in range(len(g.vs)):
	paths = g.get_all_shortest_paths(i)
	for p in paths:
		if point in p and point != p[0] and point != p[-1]:
			sp.append(p)
# 去重
spbt = 0
tu = []
for x in sp:
	if set((x[0], x[-1])) not in tu:
		tu.append(set((x[0], x[-1])))
		spbt += 1
print("betweenness =", spbt)

betweenness = 51


In [33]:
# 使用函数计算点介数
btvs = []
for p in zip(g.vs, g.betweenness()):
	btvs.append({"point": p[0]["point"], "bt": p[1]})
# key=lambda k: k["bt"]指明按照哪个指标排序
sorted(btvs, key=lambda k: k["bt"], reverse=True)

[{'bt': 51.0, 'point': '7'},
 {'bt': 51.0, 'point': '8'},
 {'bt': 22.0, 'point': '3'},
 {'bt': 22.0, 'point': '6'},
 {'bt': 22.0, 'point': '9'},
 {'bt': 22.0, 'point': '12'},
 {'bt': 0.0, 'point': '1'},
 {'bt': 0.0, 'point': '2'},
 {'bt': 0.0, 'point': '4'},
 {'bt': 0.0, 'point': '5'},
 {'bt': 0.0, 'point': '10'},
 {'bt': 0.0, 'point': '11'},
 {'bt': 0.0, 'point': '13'},
 {'bt': 0.0, 'point': '14'}]

In [6]:
# PageRank算法，计算每个节点的重要度
pg = g.pagerank()
pgvs = []
for p in zip(g.vs, pg):
	pgvs.append({"point": p[0]["point"], "pg": p[1]})
sorted(pgvs, key=lambda k: k["pg"], reverse=True)[:10]

[{'pg': 0.08621177802944507, 'point': '3'},
 {'pg': 0.08621177802944507, 'point': '6'},
 {'pg': 0.08621177802944506, 'point': '9'},
 {'pg': 0.08621177802944506, 'point': '12'},
 {'pg': 0.08311761850833196, 'point': '7'},
 {'pg': 0.08311761850833196, 'point': '8'},
 {'pg': 0.06111470635819448, 'point': '2'},
 {'pg': 0.06111470635819448, 'point': '5'},
 {'pg': 0.06111470635819448, 'point': '14'},
 {'pg': 0.06111470635819447, 'point': '1'}]